In [7]:
import pandas as pd
import numpy as np

df = pd.read_csv('../data/loan_cleaned.csv')
df.shape
df.head()

C:\Users\mahfo\AppData\Local\Temp\ipykernel_11352\2340341650.py:4: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('../data/loan_cleaned.csv')


,id,loan_amnt,term,int_rate,grade,emp_length,home_ownership,annual_inc,loan_status,purpose,addr_state,dti,fico_range_low,fico_range_high
0,68407277,3600.0,36 months,13.99,C,10+ years,MORTGAGE,55000.0,Fully Paid,debt_consolidation,PA,5.91,675.0,679.0
1,68355089,24700.0,36 months,11.99,C,10+ years,MORTGAGE,65000.0,Fully Paid,small_business,SD,16.06,715.0,719.0
2,68341763,20000.0,60 months,10.78,B,10+ years,MORTGAGE,63000.0,Fully Paid,home_improvement,IL,10.78,695.0,699.0
3,66310712,35000.0,60 months,14.85,C,10+ years,MORTGAGE,110000.0,Current,debt_consolidation,NJ,17.06,785.0,789.0
4,68476807,10400.0,60 months,22.45,F,3 years,MORTGAGE,104433.0,Fully Paid,major_purchase,PA,25.37,695.0,699.0


In [8]:
missing = df.isnull().sum()
missing_percent = (missing / len(df)) * 100

missing_df = pd.DataFrame({
    'Column': missing.index,
    'Missing Count': missing.values,
    'Missing %': missing_percent.values
})
missing_df = missing_df[missing_df['Missing Count'] > 0]
missing_df.sort_values(by='Missing %', ascending=False)

,Column,Missing Count,Missing %
5,emp_length,146940,6.499754
11,dti,1744,0.077144
7,annual_inc,37,0.001637
1,loan_amnt,33,0.001460
2,term,33,0.001460
4,grade,33,0.001460
3,int_rate,33,0.001460
8,loan_status,33,0.001460
6,home_ownership,33,0.001460
9,purpose,33,0.001460


In [19]:
invalid_loan = df[df['loan_amnt'] <= 0]
invalid_income = df[df['annual_inc'] < 0]
zero_income = df[df['annual_inc'] == 0]

fico_out_of_range = df[
    (df['fico_range_low'] < 300) | (df['fico_range_high'] > 850)
]

In [16]:
valid_terms = [' 36 months', ' 60 months']
invalid_terms = df[~df['term'].isin(valid_terms)]

valid_grades = list('ABCDEFG')
invalid_grades = df[~df['grade'].isin(valid_grades)]

In [17]:
duplicates = df[df.duplicated()]

In [20]:
checks = {
    'Zero/Negative loan amount': (df['loan_amnt'] <= 0).sum(),
    'negative income': (df['annual_inc'] < 0).sum(),
    'Zero income': (df['annual_inc'] == 0).sum(),
    'FICO < 300 or > 850':  fico_out_of_range.shape[0],
    'Invalid loan term': invalid_terms.shape[0],
    'Duplicates': duplicates.shape[0]
}

report = pd.DataFrame(list(checks.items()), columns=['Rule', 'Violations'])
report.to_csv('../reports/metrics_summary.csv', index=False)
report

,Rule,Violations
0,Zero/Negative loan amount,0
1,negative income,0
2,Zero income,1667
3,FICO < 300 or > 850,0
4,Invalid loan term,33
5,Duplicates,0


In [22]:
df_clean = df[(df['annual_inc'] > 0) & (df['fico_range_low'] >= 300) & (df['fico_range_high'] <= 850)]
df_clean.to_csv('../data/loan_cleaned_validated.csv', index=False)